In [1]:
!pip install torch torchvision

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile

zip_path   = "/content/drive/MyDrive/esort_class/imgs_sort_bin.zip"
extract_to = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_to)
print("Unpacked to:", extract_to)

Unpacked to: /content/dataset


In [4]:
import torch
from torchvision import datasets, transforms, models
from torch import nn, optim
from torch.utils.data import DataLoader

# 1) Configuration
DATA_DIR   = "/content/dataset/imgs_sort_bin"
BATCH_SIZE = 32
EPOCHS     = 25
NUM_CLASSES= 6
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# 2) Data transforms
train_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])
val_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

# 3) Datasets & loaders
train_ds = datasets.ImageFolder(f"{DATA_DIR}/train", transform=train_tf)
val_ds   = datasets.ImageFolder(f"{DATA_DIR}/val",   transform=val_tf)
train_ld = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_ld   = DataLoader(val_ds,   batch_size=BATCH_SIZE)

# 4) Model setup
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
model = model.to(DEVICE)

# 5) Optimizer & loss
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# 6) Training loop
best_acc = 0.0
for epoch in range(EPOCHS):
    # Train
    model.train()
    for imgs, labels in train_ld:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    # Validate
    model.eval()
    correct = 0
    with torch.no_grad():
        for imgs, labels in val_ld:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            preds = model(imgs).argmax(dim=1)
            correct += (preds == labels).sum().item()
    acc = correct / len(val_ds)
    print(f"Epoch {epoch+1}/{EPOCHS} — Val Acc: {acc:.4f}")
    # Save best
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "eSort_efnclass_best.pt")

print("Training complete — best validation accuracy:", best_acc)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using device: cuda
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 169MB/s]


Epoch 1/25 — Val Acc: 0.9145
Epoch 2/25 — Val Acc: 0.9355
Epoch 3/25 — Val Acc: 0.9597
Epoch 4/25 — Val Acc: 0.9629
Epoch 5/25 — Val Acc: 0.9726
Epoch 6/25 — Val Acc: 0.9661
Epoch 7/25 — Val Acc: 0.9645
Epoch 8/25 — Val Acc: 0.9629
Epoch 9/25 — Val Acc: 0.9645
Epoch 10/25 — Val Acc: 0.9629
Epoch 11/25 — Val Acc: 0.9645
Epoch 12/25 — Val Acc: 0.9629
Epoch 13/25 — Val Acc: 0.9645
Epoch 14/25 — Val Acc: 0.9742
Epoch 15/25 — Val Acc: 0.9726
Epoch 16/25 — Val Acc: 0.9758
Epoch 17/25 — Val Acc: 0.9742
Epoch 18/25 — Val Acc: 0.9726
Epoch 19/25 — Val Acc: 0.9823
Epoch 20/25 — Val Acc: 0.9806
Epoch 21/25 — Val Acc: 0.9758
Epoch 22/25 — Val Acc: 0.9742
Epoch 23/25 — Val Acc: 0.9742
Epoch 24/25 — Val Acc: 0.9758
Epoch 25/25 — Val Acc: 0.9742
Training complete — best validation accuracy: 0.9822580645161291


In [5]:
from google.colab import files
files.download("eSort_efnclass_best.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 1. Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Copy the best model to your Drive
!cp eSort_classifier_best.pt /content/drive/MyDrive/

print("Model saved to Drive at /content/drive/MyDrive/eSort-yolo/eSort_class_best.pt")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat 'eSort_classifier_best.pt': No such file or directory
Model saved to Drive at /content/drive/MyDrive/eSort-yolo/eSort_class_best.pt
